In [1]:
'''
state space
0 |Cart Position | -4.8 4.8
1 |Cart Velocity |-Inf Inf 
2 |Pole Angle | ~ -0.418 rad (-24°) ~ 0.418 rad (24°) 
3 |Pole Angular Velocity |-Inf Inf
'''

'\nstate space\n0 |Cart Position | -4.8 4.8\n1 |Cart Velocity |-Inf Inf \n2 |Pole Angle | ~ -0.418 rad (-24°) ~ 0.418 rad (24°) \n3 |Pole Angular Velocity |-Inf Inf\n'

In [4]:
Recorder?

Init signature: Recorder(*args, **kwds)
Docstring:     
save simulation as mp4
write text on a frame
Init docstring: episode_num: the number of episode a file contains
File:           ~/Documents/cart-pole/RL-for-DPIC/cart-pole-env/lib/python3.8/site-packages/gym_recorder/recorder.py
Type:           type
Subclasses:     

In [3]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import time
from gym_recorder import Recorder
from gym.wrappers.monitoring.video_recorder import VideoRecorder


class CartPoleEnv(gym.Env):

    metadata = {"render.modes": ["human", "rgb_array"], "video.frames_per_second": 50}

    def __init__(self, render_mode="rgb_array", n_observations=1000, q_table=np.array(None), alpha=0.5):
      self.env = gym.make("CartPole-v1", render_mode=render_mode)
      self.action_space = self.env.action_space
      self.observation_space = self.env.observation_space
      self.n_observation_space = 4
      self.n_buckets = 16
      self.bucket_size = self.env.observation_space.high / (self.n_buckets/2)
      self.q_table =  np.zeros((self.env.action_space.n, np.power(self.n_buckets, self.n_observation_space))) if not q_table.all() else q_table
      self.epsilon = 1
      self.n_episodes = n_observations
      self.epsilon_delta = self.epsilon / self.n_episodes
      self.metric = []
      self.alpha = alpha
      self.gamma =  0.99
      return 
    
    def get_index(self, observation):
      observation_bucketed = self.bucket_observation(observation)
      i = len(observation_bucketed) - 1
      idx = 0
      for el in observation_bucketed:
        idx = idx + (self.n_buckets**i * el)
        i -= 1
      return int(idx)
    
    def bucket_observation(self, observation):
      return(np.floor(observation/self.bucket_size))

    def update_q_table(self, observation, observation_prime, reward, action=1):
         
      Q = self.q_table[action][self.get_index(observation)]
      Q = (1-self.alpha)*Q + self.alpha*(reward + self.gamma*max(self.q_table[a][self.get_index(observation_prime)] for a in range(2))) 
      self.q_table[action][self.get_index(observation)] = Q

    def train(self):
        observation, _ = self.env.reset()
        j=0
        for i in range(self.n_episodes):
          action = self.policy(observation)
          observation_prime, reward, terminated, truncated, info = self.env.step(action)
          self.update_q_table(observation, observation_prime, reward, action)
          j +=1
          if terminated or truncated:
            self.metric.append(j)
            j=0
            observation, info = self.env.reset()
            # print(info)
        self.env.close()
      
    def test(self):
        # self.env.unwrapped.render_mode = "human"
        vid = VideoRecorder(self.env, enabled=True, path="./qlearning_test_video/vid.mp4")
        observation, _ = self.env.reset()
        for i in range(1000):
            vid.capture_frame()
            index = self.get_index(observation)
            if self.q_table[0][index] > self.q_table[1][index]:
              action = 0
            else:
              action = 1 
            observation, reward, terminated, truncated, info = self.env.step(action)
            if terminated or truncated:
              # time.sleep(0.1)
              observation, _ = self.env.reset()
        self.env.reset()
        self.env.close()
        vid.close()
            
    def policy(self, observation):
      self.epsilon = self.epsilon - np.log(self.epsilon_delta)   #at first, low probability to read from q-table, ie high prob take random action
      take_random_action = self.epsilon < np.random.random()
      if take_random_action:
         return self.action_space.sample()
      else:
         index = self.get_index(observation)
         if self.q_table[0][index] > self.q_table[1][index]:
           return 0
         else:
           return 1

In [9]:
observation = env_alpha_75.observation_space.sample()
# str([f"{observe=}" for observe in observation])
nl = '\n'
Cart_Position = observation[0]
Cart_Velocity = observation[1]
Pole_Angle =observation[2]
Pole_Angular_Velocity = observation[3]
text = f"{Cart_Position=}{nl}{Cart_Velocity=}{Pole_Angle=}{nl}{Pole_Angular_Velocity=}"
print(text)


Cart_Position=1.3460333
Cart_Velocity=2.3611106e+38Pole_Angle=-0.18353856
Pole_Angular_Velocity=-1.826864e+38


In [10]:
env_alpha_75 = CartPoleEnv(n_observations=1000, alpha=0.75)
env_alpha_75.train()
env_alpha_75.test()
plt.plot(env_alpha_75.metric)

Moviepy - Building video ./qlearning_test_video/vid.mp4.
Moviepy - Writing video ./qlearning_test_video/vid.mp4



Moviepy - Done !
Moviepy - video ready ./qlearning_test_video/vid.mp4


In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import time
from gym_recorder import Recorder
from gym.wrappers.monitoring.video_recorder import VideoRecorder


class CartPoleEnvLinear(gym.Env):

    metadata = {"render.modes": ["human", "rgb_array"], "video.frames_per_second": 50}

    def __init__(self, render_mode="rgb_array", n_observations=1000, q_table=np.array(None), alpha=0.5):
      self.env = gym.make("CartPole-v1", render_mode=render_mode)
      self.action_space = self.env.action_space
      self.observation_space = self.env.observation_space
      self.n_observation_space = 4
      self.n_buckets = 16
      self.bucket_size = self.env.observation_space.high / (self.n_buckets/2)
      self.q_table =  np.zeros((self.env.action_space.n, np.power(self.n_buckets, self.n_observation_space))) if not q_table.all() else q_table
      self.epsilon = 1
      self.n_episodes = n_observations
      self.epsilon_delta = self.epsilon / self.n_episodes
      self.metric = []
      self.alpha = alpha
      self.gamma =  0.99
      return 
    
    def get_index(self, observation):
      observation_bucketed = self.bucket_observation(observation)
      i = len(observation_bucketed) - 1
      idx = 0
      for el in observation_bucketed:
        idx = idx + (self.n_buckets**i * el)
        i -= 1
      return int(idx)
    
    def bucket_observation(self, observation):
      return(np.floor(observation/self.bucket_size))

    def update_q_table(self, observation, observation_prime, reward, action=1):
      Q = self.q_table[action][self.get_index(observation)]
      Q = (1-self.alpha)*Q + self.alpha*(reward + self.gamma*max(self.q_table[a][self.get_index(observation_prime)] for a in range(2))) 
      self.q_table[action][self.get_index(observation)] = Q

    def train(self):
        observation, _ = self.env.reset()
        j=0
        w = np.zeros(self.n_observation_space) + 1
        for i in range(self.n_episodes):
          action = self.policy(observation)
          observation_prime, reward, terminated, truncated, info = self.env.step(action)
          self.update_q_table(observation, observation_prime, reward, action)
          j +=1
          if terminated or truncated:
            self.metric.append(j)
            j=0
            observation, info = self.env.reset()
            # print(info)
        self.env.close()
      
    def test(self):
        # self.env.unwrapped.render_mode = "human"
        vid = VideoRecorder(self.env, enabled=True, path="./qlearning_test_video/vid.mp4")
        observation, _ = self.env.reset()
        for i in range(1000):
            vid.capture_frame()
            index = self.get_index(observation)
            if self.q_table[0][index] > self.q_table[1][index]:
              action = 0
            else:
              action = 1 
            observation, reward, terminated, truncated, info = self.env.step(action)
            if terminated or truncated:
              # time.sleep(0.1)
              observation, _ = self.env.reset()
        self.env.reset()
        self.env.close()
        vid.close()
            
    def policy(self, observation):
      self.epsilon = self.epsilon - np.log(self.epsilon_delta)   #at first, low probability to read from q-table, ie high prob take random action
      take_random_action = self.epsilon < np.random.random()
      if take_random_action:
         return self.action_space.sample()
      else:
         index = self.get_index(observation)
         if self.q_table[0][index] > self.q_table[1][index]:
           return 0
         else:
           return 1

In [5]:
env = CartPoleEnv()

In [8]:
np.zeros(env.n_observation_space) + 1

array([1., 1., 1., 1.])